In [71]:
import requests
import pandas as pd
from pymongo import MongoClient
import json

myclient = MongoClient("mongodb://localhost:27017/sentimental-analyze")
mydb = myclient.get_database()
col_users = mydb['users']
col_chats = mydb['chats']


In [2]:
requests.get("http://localhost:4999/help").text

'Bienvenid@!! Si tienes dudas de como usar esta API consulta el archivo README.md'

In [ ]:
# user = 'pepe'
# requests.get(f"http://localhost:4999/user/create/{user}").text

In [14]:
name_chat = 'South Park - episode 1'
new_chat = {
    'name':f'{name_chat}'
}
r = requests.get("http://localhost:4999/chat/create", params=new_chat).text

In [15]:
r

'El chat South Park - episode 1 ha sido creado con éxito'

In [5]:
df = pd.read_csv('../project_sentimental-analyze/input/All-seasons.csv')
df 

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."
...,...,...,...,...
70891,9,14,Stan,I think you're pushing it.\n
70892,9,14,Randy,How about twenty?\n
70893,9,14,Stan,That's not disciprine.\n
70894,9,14,Randy,Right right. Does vodka count?\n


In [86]:
episode_1 = df[(df['Season'] == '1') & (df['Episode'] == '1')].reset_index().drop(columns=['Season', 'Episode', 'index'])
us = set(episode_1['Character'].tolist())
tx = episode_1['Line'].tolist()
tx = [t[:-1] for t in tx]
content = list(zip(episode_1['Character'],tx))
print(content)

[('Boys', "School day, school day, teacher's golden ru..."), ('Kyle', "Ah, damn it! My little brother's trying to follow me to school again."), ('Ike', 'Zeeponanner.'), ('Kyle', "Ike, you can't come to school with me. "), ('Cartman', 'Yeah, go home you little dildo.'), ('Kyle', "Dude, don't call my brother a dildo!"), ('Stan', "What's a dildo?"), ('Kyle', "Well, I don't know...  and I'll bet Cartman doesn't know either!"), ('Cartman', 'I know what it means!'), ('Kyle', 'Well, what?'), ('Cartman', "I'm not telling you."), ('Stan', "What's a dildo, Kenny?"), ('Kenny', "(It's a giant stick that goes inside the mom's vagina) "), ('Cartman', "He-yeah, that's what Kyle's little brother is all right!  Ow! "), ('Stan', 'Dude, that kicks ass!'), ('Kyle', 'Yeah, check this one out. Ready, Ike? Kick the baby!'), ('Ike', "Don't kick the baby."), ('Kyle', 'Kick the baby. '), ('Stan', "Whoa, Cartman! Looks like you didn't get much sleep last night."), ('Cartman', "That's 'cause I was having these...

In [7]:
users = us
for user in users:
    requests.get(f"http://localhost:4999/user/create/{user}").text

In [16]:
chat = 1000
users = us
for user in users:
    user_to_chat = {
        'user':f'{user}'
    }
    requests.get(f"http://localhost:4999/chat/{chat}/adduser", params=user_to_chat).text

In [17]:
chat = 1000
messages = content
for c in content:
    text_to_chat = {
        'user':f'{c[0]}',
        'text':f'{c[1]}'
    }
    requests.get(f"http://localhost:4999/chat/{chat}/addmessage", params=text_to_chat).text

In [82]:
chat_id = 1000
requests.get(f"http://localhost:4999/chat/{chat_id}/list").json()

{'content': [{'text': "School day, school day, teacher's golden ru...",
   'user': 7},
  {'text': "Ah, damn it! My little brother's trying to follow me to school again.",
   'user': 4},
  {'text': 'Zeeponanner.', 'user': 19},
  {'text': "Ike, you can't come to school with me. ", 'user': 4},
  {'text': 'Yeah, go home you little dildo.', 'user': 10},
  {'text': "Dude, don't call my brother a dildo!", 'user': 4},
  {'text': "What's a dildo?", 'user': 11},
  {'text': "Well, I don't know...  and I'll bet Cartman doesn't know either!",
   'user': 4},
  {'text': 'I know what it means!', 'user': 10},
  {'text': 'Well, what?', 'user': 4},
  {'text': "I'm not telling you.", 'user': 10},
  {'text': "What's a dildo, Kenny?", 'user': 11},
  {'text': "(It's a giant stick that goes inside the mom's vagina) ",
   'user': 8},
  {'text': "He-yeah, that's what Kyle's little brother is all right!  Ow! ",
   'user': 10},
  {'text': 'Dude, that kicks ass!', 'user': 11},
  {'text': 'Yeah, check this one out.

In [96]:
messages = list(col_chats.find({'_id': 1000}, {
            '_id': 0, 'content': 1}))

text = messages[0]['content']
#text

m = [dictio['text'] for dictio in text]
#print(m)
m = ' '.join(m)
m

'School day, school day, teacher\'s golden ru... Ah, damn it! My little brother\'s trying to follow me to school again. Zeeponanner. Ike, you can\'t come to school with me.  Yeah, go home you little dildo. Dude, don\'t call my brother a dildo! What\'s a dildo? Well, I don\'t know...  and I\'ll bet Cartman doesn\'t know either! I know what it means! Well, what? I\'m not telling you. What\'s a dildo, Kenny? (It\'s a giant stick that goes inside the mom\'s vagina)  He-yeah, that\'s what Kyle\'s little brother is all right!  Ow!  Dude, that kicks ass! Yeah, check this one out. Ready, Ike? Kick the baby! Don\'t kick the baby. Kick the baby.  Whoa, Cartman! Looks like you didn\'t get much sleep last night. That\'s \'cause I was having these... bogus nightmares. Really? What about? Well, I dreamt that I was lying in my bed...  in the dark, when all of a sudden this bright blue light filled the room.  Then slowly my bedroom door begin to open,  and the next thing I remember, I was being drug t

In [98]:
!pip3.8 install --user -U nltk

  Using cached nltk-3.4.5.zip (1.5 MB)
    Running setup.py install for nltk ... done


In [99]:
chat_id = 1000
requests.get(f"http://localhost:4999/chat/{chat_id}/sentiment")

<Response [404]>